# Counting Vehicles from a video stream

<img src="viaduc.jpg" width=800>

## 1. Setttings

In [1]:
import sys
sys.version

'3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]'

In [2]:
#!pip install azureml-sdk

In [3]:
import cv2
import numpy as np
from time import sleep
import datetime
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

In [4]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "tobereplaced") # ID Azure
workspace_name = os.environ.get("WORKSPACE_NAME", "azuremlvision") # Azure ML Workspace name
resource_group = os.environ.get("RESOURCE_GROUP", "azuremlvision-rg") #Azure ML Resource group

In [5]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("Azure ML workspace", ws.name ,"was found: OK")
except:
   print("Error.")

Azure ML workspace azuremlvision was found: OK


In [6]:
ws = Workspace.from_config()
print('Current Azure ML Workspace: ' + ws.name, 
      'Region: ' + ws.location, 
      'Ressource Group: ' + ws.resource_group, sep='\n')

Current Azure ML Workspace: azuremlvision
Region: westeurope
Ressource Group: azuremlvision-rg


In [7]:
from azureml.core import experiment
experiment = Experiment(workspace=ws, name="Video_Traffic_Prediction")

In [8]:
run = experiment.start_logging(snapshot_directory=None)

## 2. Video Capture

In [9]:
Largeurmin=20
hauteurmin=20
offset=10
referenceline=650
delay=60 #FPS
detec = []
nbvehicles=0
traffic_per_hour=0

In [10]:
def fcentre(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

In [11]:
webcam = cv2.VideoCapture('millau.mp4')

soustrace = cv2.bgsegm.createBackgroundSubtractorMOG()
debut = datetime.datetime.now()
print("\033[1;31;34mStarting: ", debut, '\n')

while True:
    ret , frame1 = webcam.read()
    tempo = float(1/delay)
    sleep(tempo)
    
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(3,3),5)
    img_sub = soustrace.apply(blur)
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilate = cv2.morphologyEx (dilat, cv2. MORPH_CLOSE , kernel)
    dilate = cv2.morphologyEx (dilate, cv2. MORPH_CLOSE , kernel)
    contour,h=cv2.findContours(dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
      
    # Ligne de référence
    cv2.line(frame1, (10, referenceline), (1500, referenceline), (255,255,0), 3)
    
    for(i,c) in enumerate(contour):
        (x,y,w,h) = cv2.boundingRect(c)
        contourvalider = (w >= Largeurmin) and (h >= hauteurmin)
        
        if not contourvalider:
            continue
        
        # Rectangle for each vehicle
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)        
        centre = fcentre(x, y, w, h)
        detec.append(centre)
        
        # Cercle centre voiture
        cv2.circle(frame1, centre, 3, (0, 255, 0), -1)
        
        for (x,y) in detec:
            if y<(referenceline+offset) and y>(referenceline-offset):
                nbvehicles+=1
                cv2.line(frame1, (25, referenceline), (1200, referenceline), (0,255,0), 2)  
                detec.remove((x,y))
                
                now = datetime.datetime.now()
                
                # Python prints
                print(datetime.datetime.now())
                print("Number of vehicles = "+str(nbvehicles))        
    
                now2=datetime.datetime.now()
                elapsed=now2-debut
                nbseconds=elapsed.total_seconds()
                traffic_per_min=round(nbvehicles/nbseconds*60)
                traffic_per_hour=round(traffic_per_min*60)
                traffic_per_day=round(traffic_per_hour*24)
                
                print(">> Prediction of number of vehicles per minute =", str(traffic_per_min))
                print(">> Prediction of number of vehicles per hour =", str(traffic_per_hour))
                print(">> Prediction of number of vehicles per day =", str(traffic_per_day))
                print("\n", "-------------------------------------------------------------------------", "\n")
                
                # Logging values to Azure ML workspace
                run.log("Date", now)
                run.log("Number_vehicles", nbvehicles)
                run.log("Traffic_prediction_minutes", traffic_per_min)
                run.log("Traffic_prediction_hour", traffic_per_hour)
                run.log("Traffic_prediction_day", traffic_per_day)
                
    # Datetime
    maintenant = datetime.datetime.now()
    maintenant2=maintenant.strftime("%d-%b-%Y %H:%M:%S")
    
    # Displaying datetime in the video
    cv2.putText(frame1, "Counting vehicules 'Viaduc de Millau'", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0),2)
    cv2.putText(frame1, str(maintenant2), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0),2)
    
    # Displaying number of vehicules in the video
    cv2.putText(frame1, "Number of vehicles = "+ str(nbvehicles), (10, 150), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255),2)
    
    # Computing and displaying prediction traffic
    if traffic_per_hour !=0:
        cv2.putText(frame1, ">> Prediction of traffic: "+str(traffic_per_hour) 
                    + " vehicles per hour", (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255),2)
        cv2.putText(frame1, ">> Prediction of traffic: "+str(traffic_per_day) 
                    +" vehicles per day", (10, 230), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255),2)
    
    cv2.imshow("Video" , frame1)
    cv2.imshow("Resultat",dilate)

    if cv2.waitKey(1) == 27: # Press ESC to exit
        run.complete() # Stop logging Azure ML 
        break
    
cv2.destroyAllWindows()
webcam.release()

Starting:  2021-07-30 14:16:43.543972 

2021-07-30 14:16:45.043493
Number of vehicles = 1
>> Prediction of number of vehicles per minute = 40
>> Prediction of number of vehicles per hour = 2400
>> Prediction of number of vehicles per day = 57600

 ------------------------------------------------------------------------- 

2021-07-30 14:16:45.114490
Number of vehicles = 2
>> Prediction of number of vehicles per minute = 76
>> Prediction of number of vehicles per hour = 4560
>> Prediction of number of vehicles per day = 109440

 ------------------------------------------------------------------------- 

2021-07-30 14:16:46.141511
Number of vehicles = 3
>> Prediction of number of vehicles per minute = 69
>> Prediction of number of vehicles per hour = 4140
>> Prediction of number of vehicles per day = 99360

 ------------------------------------------------------------------------- 

2021-07-30 14:16:46.968513
Number of vehicles = 4
>> Prediction of number of vehicles per minute = 70
>> Pr

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-k1ohfcms\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


### Azure ML experiment

In [12]:
experiment

Name,Workspace,Report Page,Docs Page
Video_Traffic_Prediction,azuremlvision,Link to Azure Machine Learning studio,Link to Documentation


> End